In [81]:
# !pip3 install pandas
# !pip3 install seaborn
# !pip3 install plotly==5.22.0
# !pip3 install pyarrow

In [82]:
import pandas as pd
from enum import Enum
from datetime import datetime
import plotly.graph_objects as go
from pathlib import Path
import pycountry
import plotly.express as px
import requests

### CONECTAR PANDAS AO POSTGRESQL

In [83]:
class Tables(Enum):
    ACTORS = 'actors'
    DEPENDENCIES = 'dependencies'
    ISSUES = 'issues'
    METADATA = 'metadata'
    PULL_REQUESTS = 'pull_requests'
    REACTIONS = 'reactions'
    RELEASES = 'releases'
    REPOSITORIES = 'repositories'
    STARGAZERS = 'stargazers'
    TAGS = 'tags'
    TIMELINE_EVENTS = 'timeline_events'
    WATCHERS = 'watchers'

In [84]:
class Reactions(Enum):
    THUMBS_UP = 'thumbs_up'
    THUMBS_DOWN = 'thumbs_down'
    LAUGH = 'laugh'
    HOORAY = 'hooray'
    CONFUSED = 'confused'
    HEART = 'heart'
    ROCKET = 'rocket'
    EYES = 'eyes'

In [85]:
class ReactableType(Enum):
    ISSUE = 'Issue'
    PULL_REQUEST = 'PullRequest'
    PULL_REQUEST_REVIEW = 'PullRequestReview'
    PULL_REQUEST_REVIEW_COMMENT = 'PullRequestReviewComment'
    ISSUE_COMMENT = 'IssueComment'
    RELEASE = 'Release'

### LOAD DATAFRAMES

In [86]:
df = {}
for table in Tables:
    df[table] = pd.read_csv(f'./csv_backup/{table.name.lower()}.csv')

### MÉTODOS AUXILIARES

In [87]:
def keep_only_columns(data_frame, columns):
    return data_frame[columns]


def show_columns(data_frame):
    print("\n".join(list(data_frame.columns)))


def user_has_reactions(user):
    return df[Tables.REACTIONS].loc[df[Tables.REACTIONS]['user'] == user].id.count() > 0


def total_user_reactions(user):
    return df[Tables.REACTIONS].loc[df[Tables.REACTIONS]['user'] == user].id.count()


def count_user_reactions_per_type(user, reaction_type):
    return df[Tables.REACTIONS].loc[
        (df[Tables.REACTIONS]['user'] == user) & (df[Tables.REACTIONS]['content'] == reaction_type)].id.count()


def count_user_reactable_type(user, reactable_type):
    return df[Tables.REACTIONS].loc[
        (df[Tables.REACTIONS]['user'] == user) & (df[Tables.REACTIONS]['reactable_type'] == reactable_type)].id.count()


def get_author_association(user):
    author_association = df[Tables.ACTORS].loc[df[Tables.ACTORS]['id'] == user]['author_association']

    if not author_association.empty and author_association.iloc[0] != 'NONE':
        return author_association.iloc[0]

    author_association = df[Tables.ISSUES].loc[df[Tables.ISSUES]['author'] == user]['author_association']
    if author_association.empty or author_association.iloc[0] == 'NONE':
        author_association = df[Tables.PULL_REQUESTS].loc[df[Tables.PULL_REQUESTS]['author'] == user][
            'author_association']

    if author_association.empty:
        return 'NONE'
    else:
        return author_association.iloc[0]

def get_location_normalized(location):
    if location is None or location == 'None':
        return ""

    url = 'http://localhost:3000/location'
    params = {
        'q': f"{location}"
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        json = response.json()

        if json is None:
            return ""
        else:
            return json['countryName']
    return ""


def save_image(file_name, fig_to_save):
    f = Path.cwd().joinpath("images/spring_boot")
    if not f.is_dir(): f.mkdir()
    f = f.joinpath(file_name)
    fig_to_save.write_image(f, format='png', engine='kaleido')
    
    
def save_pdf(file_name, fig_to_save):
    f = Path.cwd().joinpath("images/spring_boot")
    if not f.is_dir(): f.mkdir()
    f = f.joinpath(file_name)
    fig_to_save.write_image(f, format='pdf', engine='kaleido')
    
def get_iso_alpha_3(country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except AttributeError:
        return None

### AJUSTE DOS DATAFRAMES

#### AJUSTE DO DATAFRAME ACTORS

In [88]:
# popula a quantidade de reações por tipo na tabela ACTORS e total de reacoes
df[Tables.ACTORS] = pd.read_csv(f'./csv_backup/{Tables.ACTORS.name.lower()}_processed.csv')

if df[Tables.ACTORS] is None:
    df[Tables.ACTORS] = pd.read_csv(f'./csv_backup/{Tables.ACTORS.name.lower()}.csv')
    for reaction in Reactions:
        df[Tables.ACTORS][reaction.value] = 0

    for rtype in ReactableType:
        df[Tables.ACTORS][rtype.value] = 0

    for index, row in df[Tables.ACTORS].iterrows():
        df[Tables.ACTORS].loc[index, 'author_association'] = get_author_association(row['id'])
        df[Tables.ACTORS].loc[index, 'location_n'] = get_location_normalized(row['location'])
        if user_has_reactions(row['id']):
            df[Tables.ACTORS].loc[index, 'total_reactions'] = total_user_reactions(row['id'])
            for reaction in Reactions:
                df[Tables.ACTORS].loc[index, reaction.value] = count_user_reactions_per_type(row['id'], reaction.name)
            for rtype in ReactableType:
                df[Tables.ACTORS].loc[index, rtype.value] = count_user_reactable_type(row['id'], rtype.value)
        else:
            df[Tables.ACTORS].loc[index, 'total_reactions'] = 0

    df[Tables.ACTORS].to_csv(f'./csv_backup/{Tables.ACTORS.name.lower()}_processed.csv', index=False)

#### AJUSTE DO DATAFRAME REACTIONS

In [89]:
# popula o author_association na tabela REACTIONS
df[Tables.REACTIONS] = pd.read_csv(f'./csv_backup/{Tables.REACTIONS.name.lower()}_author_association.csv')

if df[Tables.REACTIONS] is None:
    df[Tables.REACTIONS] = pd.read_csv(f'./csv_backup/{Tables.REACTIONS.name.lower()}.csv')
    for index, row in df[Tables.REACTIONS].iterrows():
        df[Tables.REACTIONS].loc[index, 'author_association'] = get_author_association(row['user'])
    df[Tables.REACTIONS].to_csv(f'./csv_backup/{Tables.REACTIONS.name.lower()}_author_association.csv', index=False)

##### BACKUP DOS DATAFRAMES

In [90]:
# for table in Tables:
#     df[table].to_csv(f'./csv_backup/{table.name.lower()}.csv', index=False)

#### INFORMAÇÕES SOBRE O REPOSITÓRIO

In [91]:
info = {'quantidade de usuarios': df[Tables.ACTORS][df[Tables.ACTORS]['type'] == 'User']['id'].count(),
        'quantidade de pull requests': df[Tables.PULL_REQUESTS]['id'].count(),
        'quantidade de estrelas do repositório': df[Tables.STARGAZERS]['user'].count(),
        'quantidade de issues do repositório': df[Tables.ISSUES]['id'].count(),
        'quantidade de reações do repositório': df[Tables.REACTIONS]['id'].count(),
        'usuarios com localizacao': df[Tables.ACTORS][df[Tables.ACTORS]['type'] == 'User']['id'].count() - df[Tables.ACTORS]['location_n'].isnull().sum(),
        'usuarios sem localizacao': df[Tables.ACTORS]['location_n'].isnull().sum()
        }

info

{'quantidade de usuarios': 93926,
 'quantidade de pull requests': 6157,
 'quantidade de estrelas do repositório': 72230,
 'quantidade de issues do repositório': 33462,
 'quantidade de reações do repositório': 31225,
 'usuarios com localizacao': 34410,
 'usuarios sem localizacao': 59516}

#### INFORMAÇÕES SOBRE AS REAÇÕES

##### QUANTIDADE DE REAÇÕES POR TIPO

In [92]:
counts = df[Tables.REACTIONS].groupby('content').size()

counts = pd.DataFrame({
    'reaction_type': counts.index,
    'count': counts.values
})

fig = px.pie(counts, values='count', names='reaction_type', title='Spring - Quantidade de reações por tipo')
fig.update_layout(width=700.0, template='plotly_white', font=dict(size=17))

##### QUANTIDADE DE REAÇÕES POR ITEM REAGÍVEL

In [93]:
counts = df[Tables.REACTIONS]['reactable_type'].value_counts()
counts = pd.DataFrame({
    'reactable_type': counts.index,
    'count': counts.values
})

fig = px.bar(counts, x='reactable_type', y='count', title='Spring - Quantidade de reações por tipo de item reagivel', template='plotly_white', width=900.0)
fig.update_layout(font=dict(size=17), xaxis_title='Item reagível', yaxis_title='Quantidade de reações')
fig.show()
# save_pdf('spring_item_reagivel.pdf', fig)

##### QUANTIDADE DE REAÇÕES POR ITEM REAGÍVEL E TIPO DE REAÇÃO

In [94]:
counts = df[Tables.REACTIONS].groupby(['reactable_type', 'content']).size()
counts = pd.DataFrame({
    'reactable_type': counts.index.get_level_values(0),
    'reaction_type': counts.index.get_level_values(1),
    'count': counts.values
})

reaction_type__unique = counts['reaction_type'].unique()
reactable_type__unique = counts['reactable_type'].unique()

categories = {
    'Issue': list(),
    'PullRequest': list(),
    'PullRequestReview': list(),
    'PullRequestReviewComment': list(),
    'IssueComment': list(),
    'Release': list()
}

fig = go.Figure()

for cat in categories:
    for r_type in reaction_type__unique:
        counts_reaction_type = counts[counts['reaction_type'] == r_type]
        counts_reaction_type_cat = counts_reaction_type[counts_reaction_type['reactable_type'] == cat]
        cat_count__values_ = counts_reaction_type_cat['count'].values[0] if not counts_reaction_type_cat.empty else 0
        categories[cat].append(cat_count__values_)
        
    fig.add_bar(x=reaction_type__unique, y=categories[cat], name=cat)
    
    
fig.update_layout(barmode="relative", title='Spring - Proporção de reações por tipo de item reagível e tipo de reação', barnorm="percent",
                  xaxis_title='Tipo de item reagível', yaxis_title='quantidade de reações', template='plotly_white', width=900.0, font=dict(size=17))

fig.show()
# save_pdf('spring_reactable_type_reaction_type_relative.pdf', fig)

In [95]:
counts = df[Tables.REACTIONS].groupby(['reactable_type', 'content']).size()
counts = pd.DataFrame({
    'reactable_type': counts.index.get_level_values(0),
    'reaction_type': counts.index.get_level_values(1),
    'count': counts.values
})

reaction_type__unique = counts['reaction_type'].unique()
reactable_type__unique = counts['reactable_type'].unique()

categories = {
    'Issue': list(),
    'PullRequest': list(),
    'PullRequestReview': list(),
    'PullRequestReviewComment': list(),
    'IssueComment': list(),
    'Release': list()
}

fig = go.Figure()

for cat in categories:
    for r_type in reaction_type__unique:
        counts_reaction_type = counts[counts['reaction_type'] == r_type]
        counts_reaction_type_cat = counts_reaction_type[counts_reaction_type['reactable_type'] == cat]
        cat_count__values_ = counts_reaction_type_cat['count'].values[0] if not counts_reaction_type_cat.empty else 0
        categories[cat].append(cat_count__values_)
        
    fig.add_bar(x=reaction_type__unique, y=categories[cat], name=cat)
    
    
fig.update_layout(barmode="group", title='Spring - Quantidade de reações por tipo de item reagível e tipo de reação',
                  xaxis_title='Tipo de item reagível', yaxis_title='quantidade de reações', template='plotly_white', width=900.0, font=dict(size=17))

fig.show()
# save_pdf('spring_reactable_type_reaction_type_abs.pdf', fig)

In [96]:
# periodo 01/04/2021 - 01/07/2021
counts = df[Tables.REACTIONS].loc[(df[Tables.REACTIONS]['created_at'] >= '2021-04-01') & (df[Tables.REACTIONS]['created_at'] <= '2021-07-01')].groupby(['reactable_type', 'content']).size()

counts = pd.DataFrame({
    'reactable_type': counts.index.get_level_values(0),
    'reaction_type': counts.index.get_level_values(1),
    'count': counts.values
})

reaction_type__unique = counts['reaction_type'].unique()
reactable_type__unique = counts['reactable_type'].unique()

categories = {
    'Issue': list(),
    'PullRequest': list(),
    'PullRequestReview': list(),
    'PullRequestReviewComment': list(),
    'IssueComment': list(),
    'Release': list()
}

fig = go.Figure()

for cat in categories:
    for r_type in reaction_type__unique:
        counts_reaction_type = counts[counts['reaction_type'] == r_type]
        counts_reaction_type_cat = counts_reaction_type[counts_reaction_type['reactable_type'] == cat]
        cat_count__values_ = counts_reaction_type_cat['count'].values[0] if not counts_reaction_type_cat.empty else 0
        categories[cat].append(cat_count__values_)
        
    fig.add_bar(x=reaction_type__unique, y=categories[cat], name=cat)
    
    
fig.update_layout(barmode="relative", title='Spring - Reações no período de 01/04/2021 - 01/07/2021',
                  xaxis_title='Tipo de item reagível', yaxis_title='quantidade de reações', template='plotly_white', width=900.0, font=dict(size=17))

fig.show()

##### PROPORÇÃO DE REAÇÕES POR ITEM REAGÍVEL E TIPO DE USUÁRIO

In [97]:
reactable_type = ['Issue', 'PullRequest', 'PullRequestReview', 'PullRequestReviewComment', 'IssueComment', 'Release']

categories = {
    'NONE': list(),
    'CONTRIBUTOR': list(),
    'MEMBER': list()
}

fig = go.Figure()

for cat in categories:
    for r_type in reactable_type:
        cat_ = df[Tables.REACTIONS][df[Tables.REACTIONS]['author_association'] == cat]
        df_type_ = cat_[cat_['reactable_type'] == r_type]
        categories[cat].append(df_type_.size)

    fig.add_bar(x=reactable_type, y=categories[cat], name=cat)

fig.update_layout(barmode="relative", barnorm="percent" ,title='quantidade reações por tipo de item reagível separado por tipo de membro',
                  xaxis_title='Tipo de item reagível', yaxis_title='% de reações', template='plotly_white', width=900.0, font=dict(size=17))
fig.show()
# save_pdf('spring_reactions_reactable_type_member.pdf', fig)

##### PROPORÇÃO TOTAL DE REAÇÕES POR TIPO DE USUÁRIO

In [98]:
frame = pd.DataFrame(df[Tables.REACTIONS]['author_association'].value_counts(), columns=['author_association'])
fig = px.pie(frame, values='author_association', names=frame.index, title='Spring - Quantidade de reações por tipo de membro')
fig.update_layout(width=700.0, template='plotly_white', font=dict(size=17))
fig.update_traces(texttemplate="%{percent:.2%}")
fig.show()
# save_pdf('spring_reactions_member_type.pdf', fig)

##### QUANTIDADE DE REAÇÕES POR LOCALIZAÇÃO DOS USUÁRIOS

In [99]:
df_only_reactions = df[Tables.ACTORS].loc[df[Tables.ACTORS]['total_reactions'] != 0.0]

counts = df_only_reactions.groupby('location_n').size().sort_values(ascending=False)[:10]

counts = pd.DataFrame({
    'location': counts.index,
    'count': counts.values
})

fig = px.bar(counts, x='location', y='count', title='Spring - Quantidade de usuários com reações por localização', template='plotly_white', width=900.0)
fig.update_layout(font=dict(size=17), xaxis_title='Localização', yaxis_title='Quantidade de usuários')
fig.show()
# save_pdf('spring-boot_reactions_locations.pdf', fig)

##### QUANTIDADE DE REAÇÕES POR MÊS

In [100]:
for index, row in df[Tables.REACTIONS].iterrows():
    date_formated = datetime.fromisoformat(str(row['created_at'])).strftime('%Y-%m-1')
    df[Tables.REACTIONS].loc[index, 'date'] = date_formated

df_reactions = df[Tables.REACTIONS]
df_reactions = pd.DataFrame(df_reactions.groupby(['date']).size().sort_index(ascending=True),
                            columns=['count'])

scatter = go.Scatter(x=df_reactions.index, y=df_reactions['count'])
fig = go.Figure([scatter])
fig.update_layout(title = 'Quantidade de reações por mês', xaxis_title='Mês', width=900.0, yaxis_title='Quantidade de reações', template='plotly_white', font=dict(size=17))
fig.update_xaxes(tickformat="%Y-%B", dtick="M4")
fig.show()
# save_pdf('spring-boot_reactions_temporal_series.pdf', fig)

##### PROPORÇÃO DAS REAÇÕES POR TIPO DE REAÇÃO E TIPO DE USUÁRIO

In [101]:
reactions = ['THUMBS_UP', 'HEART', 'HOORAY', 'ROCKET', 'EYES', 'LAUGH', 'THUMBS_DOWN', 'CONFUSED']

categories = {
    'NONE': list(),
    'CONTRIBUTOR': list(),
    'MEMBER': list(),
}

fig = go.Figure()

for cat in categories:
    for reaction in reactions:
        cat_ = df_only_reactions[df_only_reactions['author_association'] == cat]
        categories[cat].append(cat_[reaction.lower()].sum())

    fig.add_bar(x=reactions, y=categories[cat], name=cat)

fig.update_layout(barmode="relative", barnorm="percent" ,title='Quantidade de reações por tipo de reação e tipo de membro',
                  xaxis_title='Tipo de reação', yaxis_title='% de reações', template='plotly_white', width=900.0, font=dict(size=17))
fig.show()
# save_pdf('spring_reactions_type_member_type.pdf', fig)

#### INFORMAÇÕES SOBRE OS USUÁRIOS

In [102]:
df_only_reactions = df[Tables.ACTORS].loc[df[Tables.ACTORS]['total_reactions'] != 0.0]
info = {'quantidade de usuários': df[Tables.ACTORS]['id'].count(),
        'quantidade de usuários com reações': df_only_reactions['id'].count(),
        'quantidade de usuários sem reações': df[Tables.ACTORS]['id'].count() - df_only_reactions['id'].count(),
        'quantidade de usuários com reações por tipo': df_only_reactions['author_association'].value_counts().to_dict()
        }
info

{'quantidade de usuários': 94050,
 'quantidade de usuários com reações': 14518,
 'quantidade de usuários sem reações': 79532,
 'quantidade de usuários com reações por tipo': {'NONE': 13899,
  'CONTRIBUTOR': 597,
  'MEMBER': 22}}

##### QUANTIDADE DE USUÁRIOS COM REAÇÕES

In [103]:
df_only_reactions = df[Tables.ACTORS].loc[df[Tables.ACTORS]['total_reactions'] != 0.0]

counts = {
    'usuários com reações': df_only_reactions['id'].count(),
    'usuários sem reações': df[Tables.ACTORS]['id'].count() - df_only_reactions['id'].count()
}

fig = px.pie(values=list(counts.values()), names=list(counts.keys()), title='Spring - Quantidade de usuários com reações')
fig.update_layout(width=700.0, template='plotly_white', font=dict(size=17))
fig.show()

##### LOCALIZAÇÕES COM MAIS USUÁRIOS

In [104]:
counts = df[Tables.ACTORS].groupby('location_n').size().sort_values(ascending=False)[:10]

counts = pd.DataFrame({
    'location': counts.index,
    'count': counts.values
})

fig = px.bar(counts, x='location', y='count', title='Spring - Quantidade de usuários por localização', template='plotly_white', width=900.0)
fig.update_layout(font=dict(size=17), xaxis_title='Localização', yaxis_title='Quantidade de usuários')
fig.show()
# save_pdf('spring-boot_user_locations.pdf', fig)

##### QUANTIDADE DE USUÁRIOS DAS 10 LOCALIZACOES COM MAIS USUÁRIOS

In [105]:
counts = df[Tables.ACTORS].groupby('location_n').size().sort_values(ascending=False)[:10]

counts = pd.DataFrame({
    'location': counts.index,
    'count': counts.values
})

counts['count'].sum()

23567